# Calendar Module Documentation

This notebook documents the `Calendar`, `Event`, and `Category` models and their serializers in the `apps.calendar` module.

---

## Models Overview

### 1. BaseModel (Base)
Abstract base class providing automatic UUIDs and timestamps for all calendar models.

| Field | Type | Description |
|-------|------|-------------|
| `ukid` | UUIDField | Unique identifier (auto-generated, indexed) |
| `created_at` | DateTimeField | Automatically set when record is created |
| `created_by` | IntegerField | User ID who created the record (nullable) |
| `updated_at` | DateTimeField | Automatically updated on every save |
| `updated_by` | IntegerField | User ID who last updated the record (nullable) |

---

### 2. Calendar
Represents a calendar entity defined by a title and a date range (e.g., "Academic Year 2024-2025").

| Field | Type | Constraints | Description |
|-------|------|-------------|-------------|
| `id` | AutoField | Primary Key | Auto-generated unique identifier |
| `ukid` | UUIDField | Unique, Indexed | Inherited from BaseModel |
| `title` | CharField | max_length=200, Required | Name of the calendar |
| `start_date` | DateField | Required | Start date of the calendar period |
| `end_date` | DateField | Required | End date of the calendar period |
| `created_at` | DateTimeField | auto_now_add | Inherited from BaseModel |
| `created_by` | IntegerField | Optional | Inherited from BaseModel |
| `updated_at` | DateTimeField | auto_now | Inherited from BaseModel |
| `updated_by` | IntegerField | Optional | Inherited from BaseModel |

**Relationships:**
- Has many `Event` via `events` related_name

**Ordering:** `['start_date']`

---

### 3. Category
Represents a category for events (e.g., Work, Personal, Holiday, Exam).

| Field | Type | Constraints | Description |
|-------|------|-------------|-------------|
| `id` | AutoField | Primary Key | Auto-generated unique identifier |
| `ukid` | UUIDField | Unique, Indexed | Inherited from BaseModel |
| `name` | CharField | max_length=100, Required | Category name |
| `color` | CharField | max_length=7, Optional | Hex color code for UI display (e.g., `#FF5733`) |
| `description` | TextField | Optional | Detailed description of the category |
| `created_at` | DateTimeField | auto_now_add | Inherited from BaseModel |
| `created_by` | IntegerField | Optional | Inherited from BaseModel |
| `updated_at` | DateTimeField | auto_now | Inherited from BaseModel |
| `updated_by` | IntegerField | Optional | Inherited from BaseModel |

**Relationships:**
- Has many `Event` via `events` related_name
- **PROTECTED**: Cannot delete a category that has associated events

**Ordering:** `['name']`

---

### 4. Event
Represents an event within a calendar or independently.

| Field | Type | Constraints | Description |
|-------|------|-------------|-------------|
| `id` | AutoField | Primary Key | Auto-generated unique identifier |
| `ukid` | UUIDField | Unique, Indexed | Inherited from BaseModel |
| `category` | ForeignKey | **Required**, on_delete=PROTECT | Event category - cannot delete category with events |
| `calendar` | ForeignKey | Optional, on_delete=SET_NULL | Associated calendar (can be unlinked) |
| `title` | CharField | max_length=200, Required | Event title |
| `description` | TextField | Optional | Detailed event description |
| `organizer` | CharField | max_length=100, Optional | Name of person/team organizing the event |
| `location` | CharField | max_length=255, Optional | Physical or virtual location |
| `slug` | SlugField | Unique, Auto-generated | URL-friendly identifier from title |
| `type` | CharField | max_length=10, Required | Event type (see choices below) |
| `start_date` | DateField | Required* | Event start date |
| `end_date` | DateField | Required* | Event end date |
| `start_time` | TimeField | **Required** | First Day start time |
| `end_time` | TimeField | **Required** | Last Day end time |
| `event_duration` | DurationField | Optional | Duration of event each day (for multi-day) |
| `entry_form_required` | BooleanField | default=False | Whether event requires registration |
| `registration_url` | URLField | Optional | URL for event registration |
| `registration_limit` | PositiveIntegerField | Optional | Maximum number of registrations |
| `reminder_enabled` | BooleanField | default=False | Whether to send reminders |
| `remainder_time_before_event` | DurationField | Optional | When to send reminder before event |
| `status` | CharField | max_length=20, default='draft' | Publication status (see choices below) |
| `published_at` | DateTimeField | Optional | When event was published |
| `published_by` | IntegerField | Optional | User ID who published the event |
| `postponed_to` | DateField | Optional | New date if event is postponed |
| `cancelled_at` | DateTimeField | Optional | When event was cancelled |
| `cancelled_by` | IntegerField | Optional | User ID who cancelled the event |
| `created_at` | DateTimeField | auto_now_add | Inherited from BaseModel |
| `created_by` | IntegerField | Optional | Inherited from BaseModel |
| `updated_at` | DateTimeField | auto_now | Inherited from BaseModel |
| `updated_by` | IntegerField | Optional | Inherited from BaseModel |

**Type Choices:**
| Value | Constant | Label | Description |
|-------|----------|-------|-------------|
| `single` | `SINGLE_DAY` | Single Day Event | Event occurs on one day only |
| `multi` | `MULTI_DAY` | Multi-day Event | Event spans multiple days |

**Status Choices:**
| Value | Constant | Label | Description |
|-------|----------|-------|-------------|
| `draft` | `DRAFT` | Draft | Event is not publicly visible |
| `published` | `PUBLISHED` | Published | Event is publicly visible |
| `postponed` | `POSTPONED` | Postponed | Event has been postponed |
| `cancelled` | `CANCELLED` | Cancelled | Event has been cancelled |

**Ordering:** `['start_date', 'start_time']`

**Indexes:**
- `['category', 'status']`
- `['calendar', 'start_date']`

---

## API Serializers

The Calendar module follows a three-serializer pattern for each model:

### Calendar Serializers

**CalendarCreateSerializer** - For POST operations
- Fields: `title`, `start_date`, `end_date`
- Validation: `end_date` must be after `start_date`

**CalendarUpdateSerializer** - For PUT/PATCH operations
- Fields: `title`, `start_date`, `end_date`, `updated_by` (all optional)
- Validation: Same as Create but validates only provided fields

**CalendarResponseSerializer** - For GET operations
- Fields: All model fields plus computed `event_count`
- Nested: No nested objects
- Read-only: `ukid`, `created_at`, `updated_at`

### Category Serializers

**CategoryCreateSerializer** - For POST operations
- Fields: `name`, `color`, `description`
- Validation: Color must be 7-character hex code starting with '#'

**CategoryUpdateSerializer** - For PUT/PATCH operations
- Fields: `name`, `color`, `description`, `updated_by` (all optional)
- Validation: Same color validation as Create

**CategoryResponseSerializer** - For GET operations
- Fields: All model fields plus computed `event_count`
- Nested: No nested objects
- Read-only: `ukid`, `created_at`, `updated_at`

### Event Serializers

**EventCreateSerializer** - For POST operations
- Fields: `category`, `calendar`, `title`, `description`, `organizer`, `location`, `type`, `start_date`, `end_date`, `start_time`, `end_time`, `event_duration`, `entry_form_required`, `registration_url`, `registration_limit`, `reminder_enabled`, `remainder_time_before_event`
- Validation: 
  - `end_time` > `start_time`
  - Multi-day: `end_date` > `start_date`
  - Single-day: `start_date` == `end_date`
  - If `entry_form_required`: `registration_url` is required

**EventUpdateSerializer** - For PUT/PATCH operations  
- Fields: All Create fields plus `status`, `postponed_to`, `updated_by` (all optional)
- Auto-set behavior:
  - Status → 'published': Sets `published_at` and `published_by`
  - Status → 'cancelled': Sets `cancelled_at` and `cancelled_by`

**EventResponseSerializer** - For GET operations
- Fields: All model fields
- Nested: Full `category` and `calendar` objects
- Computed: `type_display`, `status_display`
- Read-only: `ukid`, `slug`, `published_at`, `published_by`, `cancelled_at`, `cancelled_by`, `created_at`, `updated_at`

---

## Usage Examples

```python
from apps.calendar.models import Calendar, Category, Event
from datetime import date, time, timedelta

# Create a Category
work_cat = Category.objects.create(
    name="Work",
    color="#0000FF",
    description="Work-related events"
)

# Create a Calendar
cal = Calendar.objects.create(
    title="Academic Year 2025",
    start_date=date(2025, 1, 1),
    end_date=date(2025, 12, 31)
)

# Create a Single Day Event (Linked to Calendar)
event_single = Event.objects.create(
    calendar=cal,
    category=work_cat,
    title="Team Meeting",
    type=Event.SINGLE_DAY,
    start_date=date(2025, 1, 10),
    end_date=date(2025, 1, 10),  # Same as start_date for single-day
    start_time=time(10, 0),
    end_time=time(11, 0),
    status=Event.EVENT_STATUS_PUBLISHED
)

# Create a Multi-day Event
event_multi = Event.objects.create(
    calendar=cal,
    category=work_cat,
    title="Annual Conference",
    type=Event.MULTI_DAY,
    start_date=date(2025, 3, 1),
    end_date=date(2025, 3, 3),  # Must be after start_date
    start_time=time(9, 0),
    end_time=time(17, 0),
    event_duration=timedelta(hours=8),
    location="Conference Hall A",
    organizer="HR Department"
)

# Create an Independent Event (No Calendar)
event_independent = Event.objects.create(
    calendar=None,  # Can be linked later
    category=work_cat,
    title="Unassigned Task",
    type=Event.SINGLE_DAY,
    start_date=date(2025, 3, 15),
    end_date=date(2025, 3, 15),
    start_time=time(14, 0),
    end_time=time(15, 0)
)
```

---

## Validation Rules

The `Event.clean()` method enforces the following validations:

### Time Validation
1. Both `start_time` and `end_time` are **required**
2. `end_time` must be after `start_time`

### Date Validation for Single-Day Events (`type='single'`)
1. Both `start_date` and `end_date` are required
2. `start_date` and `end_date` **must be the same**

### Date Validation for Multi-Day Events (`type='multi'`)
1. Both `start_date` and `end_date` are required
2. `end_date` must be **after** `start_date`

### Slug Auto-generation
- Generated from `title` using `slugify()`
- If slug exists, appends counter (e.g., `team-meeting-1`, `team-meeting-2`)

---

## Edge Cases

### Event Operations
| Scenario | Result |
|----------|--------|
| Create event with `end_time <= start_time` | ❌ ValidationError |
| Create multi-day event without `end_date` | ❌ ValidationError |
| Create single-day event with different dates | ❌ ValidationError |
| Delete Category with associated events | ❌ ProtectedError |
| Delete Calendar with events | ✅ Events remain (calendar set to NULL) |
| Create event with same title | ✅ Slug auto-incremented |

### Calendar Operations
| Scenario | Result |
|----------|--------|
| Create calendar with `end_date < start_date` | ⚠️ No validation (allowed) |
| Create overlapping calendars | ⚠️ No validation (allowed) |

---

## Improvement Suggestions

### High Priority

1. **Add Calendar date validation**
   ```python
   def clean(self):
       if self.end_date <= self.start_date:
           raise ValidationError('End date must be after start date')
   ```

2. **Add Event date range validation**
   - Ensure event dates fall within associated calendar's date range

3. **Add unique constraint for Calendar**
   - Prevent duplicate calendar titles for overlapping periods

4. **Add Category unique name**
   - Consider making `name` unique to prevent duplicates

### Medium Priority

5. **Add soft delete for Events**
   - Use `is_deleted` flag instead of hard delete

6. **Add recurring events support**
   - Weekly/monthly/yearly repeating events
   - Recurrence rules (RRULE format)

7. **Add event attendees**
   - ManyToMany relationship with User model
   - RSVP status tracking

8. **Add event attachments**
   - File uploads for event materials

### Low Priority

9. **Add calendar visibility levels**
   - Public/private/restricted access

10. **Add event notifications**
    - Integration with email/push notifications

11. **Add timezone support**
    - Store and display times in user's timezone

12. **Add event templates**
    - Reusable event configurations

### API Improvements

13. **Add bulk event creation** - Multiple events at once
14. **Add event search** - Full-text search on title/description
15. **Add calendar export** - iCal/Google Calendar export
16. **Add conflict detection** - Warn when events overlap

---

## Database Optimization

Currently no custom indexes are defined. Consider adding:

```python
class Event(BaseModel):
    class Meta:
        indexes = [
            models.Index(fields=['start_date', 'end_date']),
            models.Index(fields=['status']),
            models.Index(fields=['calendar', 'start_date']),
            models.Index(fields=['category']),
        ]
```

### Query Optimization
- Use `select_related('category', 'calendar')` when fetching events
- Use date range queries for calendar views
- Consider caching frequently accessed categories